In [ ]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

parameters = {'axes.grid': True}
plt.rcParams.update(parameters)

import optuna
from optuna.samplers import TPESampler
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import mean_squared_error

In [ ]:
df_train = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv')
df_test = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv')

In [ ]:
df_train.head()

In [ ]:
# Removing 'id' column

df_train.drop('id', axis = 1, inplace = True)

In [ ]:
# Checking 'nulls'

df_train.isnull().sum().max() == 0

In [ ]:
df_test.head()

In [ ]:
# Removing 'id' column

df_test.drop('id', axis = 1, inplace = True)

In [ ]:
# Checking 'nulls'

df_test.isnull().sum().max() == 0

In [ ]:
# Plotting some graphs of random features in train set 

fig = plt.figure(figsize = (15, 10))
for j in [j for j in range(1, 16)]:
    i = np.random.randint(0, df_train.columns.size - 1)
    plt.subplot(3, 5, j)
    sns.kdeplot(x = df_train[df_train.columns[i]])
    plt.title(df_train.columns[i])
fig.tight_layout()
print('15 graphs of random features in train set')
plt.show()

In [ ]:
# Plotting graph of target

plt.figure(figsize = (15, 5))
sns.histplot(x = df_train['loss'], kde = True)
plt.title('Distribution of target (loss)')

In [ ]:
df_train_no_target = df_train.drop('loss', axis = 1)

In [ ]:
# Standarization

scaler = StandardScaler()

df_train_no_target_scal = pd.DataFrame(scaler.fit_transform(df_train_no_target), columns = df_train_no_target.columns)
df_test = pd.DataFrame(scaler.fit_transform(df_test), columns = df_test.columns)

In [ ]:
df_train_no_target_scal.head()

In [ ]:
df_test.head()

In [ ]:
X = df_train_no_target_scal
y = df_train['loss']

In [ ]:
# Division data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 15)

In [ ]:
# Prediction by default settings

model = xgb.XGBRegressor(objective = 'reg:tweedie', tree_method = 'gpu_hist', predictor = 'gpu_predictor', 
                         sampling_method = 'gradient_based', max_bin = 512, silent = False, random_state = 17)
model.fit(X_train, y_train)
preds_test = model.predict(X_test)
preds_train = model.predict(X_train)

In [ ]:
# RMSE score

print(f" Test RMSE score:     {np.sqrt(mean_squared_error(y_test, preds_test))}")
print(f" Train RMSE score:    {np.sqrt(mean_squared_error(y_train, preds_train))}")

In [ ]:
def check_model(model = model, n_splits = 10):
    
    scores= []
    cv = KFold(n_splits, shuffle = True)
    
    for train_idx, test_idx in cv.split(X):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        score = np.sqrt(mean_squared_error(y_test, preds))
        scores.append(score)
        
    print('************************************')    
    print(f"Mean RMSE score:       {np.mean(scores)}")
    print(f"Std RMSE:              {np.std(scores)}")

In [ ]:
# Mean RSME score 

check_model(model)

In [ ]:
def objective(trial):
    params = {'tweedie_variance_power': trial.suggest_discrete_uniform('tweedie_variance_power', 1.03, 1.12, 0.0001),
              'n_estimators': trial.suggest_int('n_estimators', 3500, 5000), 
              'max_depth': trial.suggest_int('max_depth', 4, 8),
              'eta': trial.suggest_float('eta', 0.005, 0.012),
              'subsample': trial.suggest_discrete_uniform('subsample', 0.3, 0.6, 0.01),
              'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.4, 1.0, 0.01),
              'colsample_bylevel': trial.suggest_discrete_uniform('colsample_bylevel', 0.5, 0.9, 0.01),
              'colsample_bynode': trial.suggest_discrete_uniform('colsample_bynode', 0.2, 1.0, 0.01),
              'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-3, 1),
              'reg_alpha': trial.suggest_loguniform('reg_alpha', 1, 1e2),
              'reg_lambda': trial.suggest_float('reg_lambda', 1000, 10000),
              'max_delta_step': trial.suggest_loguniform('max_delta_step', 1e-1, 1e4),
              'gamma': trial.suggest_loguniform('gamma', 1e-3, 1),
              'base_score': trial.suggest_float('base_score', 0.42, 0.48)}
        
    model = xgb.XGBRegressor(objective = 'reg:tweedie', tree_method = 'gpu_hist', predictor = 'gpu_predictor',
                             sampling_method = 'gradient_based', n_jobs = -1, max_bin = 512,
                             **params, random_state = 17)
    
    scores= []
    cv = KFold(n_splits = 10, shuffle = True)
    
    for train_idx, test_idx in cv.split(X):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]
        
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        
        score = np.sqrt(mean_squared_error(y_test, preds))
        scores.append(score / cv.n_splits)
    
    return sum(scores)

In [ ]:
# Improving parameters by optuna

study = optuna.create_study(direction = "minimize", sampler = TPESampler())
study.optimize(objective, n_trials = 30)

In [ ]:
params = study.best_params
params

In [ ]:
# Mean RSME score with improved parameters

check_model(model = xgb.XGBRegressor(objective = 'reg:tweedie', tree_method = 'gpu_hist', predictor = 'gpu_predictor', 
                                     sampling_method = 'gradient_based', n_jobs=-1, max_bin = 512,
                                     random_state = 17, silent = False, **params))

In [ ]:
model = xgb.XGBRegressor(objective = 'reg:tweedie', tree_method = 'gpu_hist', predictor = 'gpu_predictor',
                         sampling_method = 'gradient_based', n_jobs = -1, max_bin = 512,
                         random_state = 17, silent = False, **params)
model.fit(X, y)
preds = model.predict(df_test)
preds = preds.astype(int)

In [ ]:
sub = pd.read_csv('../input/tabular-playground-series-aug-2021/sample_submission.csv')
sub['loss'] = preds
sub.sample(5)

In [ ]:
sub.to_csv('XGB+Optuna.csv', index = False)